In [1]:
import numpy as np
import os
import argparse
import time
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.transforms as trn
import torchvision.datasets as dset
import torch.nn.functional as F
from tqdm.notebook import tqdm
from utils.wrn import WideResNet
import utils.attacks as attacks
from utils.detector import Detector, style_loss
from tqdm.notebook import tqdm

## Args

In [2]:
parser = argparse.ArgumentParser(description='Trains a CIFAR Classifier',
                                 formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('--dataset', '-d', type=str, default='cifar10', choices=['cifar10', 'cifar100'],
                    help='Choose between CIFAR-10, CIFAR-100.')
parser.add_argument('--model', '-m', type=str, default='wrn',
                    choices=['allconv', 'wrn'], help='Choose architecture.')
# Optimization options
parser.add_argument('--epochs', '-e', type=int, default=100, help='Number of epochs to train.')
parser.add_argument('--learning_rate', '-lr', type=float, default=0.1, help='The initial learning rate.')
parser.add_argument('--batch_size', '-b', type=int, default=128, help='Batch size.')
parser.add_argument('--test_bs', type=int, default=256)
parser.add_argument('--momentum', type=float, default=0.9, help='Momentum.')
parser.add_argument('--decay', type=float, default=0.0005, help='Weight decay (L2 penalty).')
# WRN Architecture
parser.add_argument('--layers', default=40, type=int, help='total number of layers')
parser.add_argument('--widen-factor', default=2, type=int, help='widen factor')
parser.add_argument('--droprate', default=0.0, type=float, help='dropout probability')
# Checkpoints
parser.add_argument('--save', '-s', type=str, default='./snapshots/baseline', help='Folder to save checkpoints.')
parser.add_argument('--load', '-l', type=str, default='', help='Checkpoint path to resume / test.')
parser.add_argument('--test', '-t', action='store_true', help='Test only flag.')
# Acceleration
parser.add_argument('--ngpu', type=int, default=1, help='0 = CPU.')
parser.add_argument('--gpu', type=int, default=1, help='0 = CPU.')
parser.add_argument('--prefetch', type=int, default=2, help='Pre-fetching threads.')
args = parser.parse_args(["--save", "checkpoints_style/", "--gpu", "3", "--test_bs", "128", "-b", "256", "-e", "300"])

## Initialization

In [3]:
state = {k: v for k, v in args._get_kwargs()}
print(state)

torch.manual_seed(1)
np.random.seed(1)

# # mean and standard deviation of channels of CIFAR-10 images
# mean = [x / 255 for x in [125.3, 123.0, 113.9]]
# std = [x / 255 for x in [63.0, 62.1, 66.7]]

train_transform = trn.Compose([trn.RandomHorizontalFlip(), trn.RandomCrop(32, padding=4),
                               trn.ToTensor()])
test_transform = trn.Compose([trn.ToTensor()])

train_data = dset.CIFAR10('~/datasets/cifarpy', train=True, transform=train_transform, download=True)
test_data = dset.CIFAR10('~/datasets/cifarpy', train=False, transform=test_transform, download=True)
num_classes = 10

train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=args.batch_size, shuffle=True,
    num_workers=args.prefetch, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=args.test_bs, shuffle=False,
    num_workers=args.prefetch, pin_memory=True)



normalize = trn.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
detector_data_transform = trn.Compose([trn.ToTensor(), normalize])

data_train = list(torch.utils.data.DataLoader(
        dset.CIFAR10('~/datasets/cifarpy', 
                     train=True, 
                     transform=detector_data_transform, 
                     download=True),
        batch_size=1, shuffle=False))

data_test = list(torch.utils.data.DataLoader(
        dset.CIFAR10('~/datasets/cifarpy', 
                     train=False, 
                     transform=detector_data_transform, 
                     download=True),
        batch_size=1, shuffle=False))


{'save': 'checkpoints_style/', 'decay': 0.0005, 'dataset': 'cifar10', 'learning_rate': 0.1, 'prefetch': 2, 'widen_factor': 2, 'droprate': 0.0, 'test': False, 'ngpu': 1, 'momentum': 0.9, 'gpu': 3, 'model': 'wrn', 'test_bs': 128, 'layers': 40, 'load': '', 'batch_size': 256, 'epochs': 300}
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
# Create model
if args.model == 'allconv':
    net = AllConvNet(num_classes)
else:
    net = WideResNet(args.layers, num_classes, args.widen_factor, dropRate=args.droprate)
#     net.load_state_dict(torch.load("benchmark_ckpts/cifar10_style_epoch_99.pt"))

start_epoch = 0

# Restore model if desired
if args.load != '':
    for i in range(1000 - 1, -1, -1):
        model_name = os.path.join(args.load, args.dataset + '_' + args.model +
                                  '_baseline_epoch_' + str(i) + '.pt')
        if os.path.isfile(model_name):
            net.load_state_dict(torch.load(model_name))
            print('Model restored! Epoch:', i)
            start_epoch = i + 1
            break

    if start_epoch == 0:
        assert False, "could not resume"

if args.ngpu > 1:
    net = torch.nn.DataParallel(net, device_ids=list(range(args.ngpu)))

if args.ngpu > 0:
    torch.cuda.set_device(args.gpu)
    net.cuda()
    torch.cuda.manual_seed(1)

# cudnn.benchmark = True  # fire on all cylinders

optimizer = torch.optim.SGD(
    net.parameters(), state['learning_rate'], momentum=state['momentum'],
    weight_decay=state['decay'], nesterov=True)

def cosine_annealing(step, total_steps, lr_max, lr_min):
    return lr_min + (lr_max - lr_min) * 0.5 * (
            1 + np.cos(step / total_steps * np.pi))


scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer,
    lr_lambda=lambda step: cosine_annealing(
        step,
        args.epochs * len(train_loader),
        1,  # since lr_lambda computes multiplicative factor
        1e-6 / args.learning_rate))


adversary = attacks.PGD(epsilon=8./255, num_steps=10, step_size=2./255).cuda()

## Training

In [5]:
def train():
    net.train()  # enter train mode
    loss_avg, loss_gram_avg = 0.0, 0.0
    
    for bx, by in tqdm(train_loader):
        bx, by = bx.cuda(), by.cuda()

        adv_bx = adversary(net, bx, by)
                        
        # forward
        logits_reg, feats_reg = net.gram_forward(bx * 2 - 1)
        logits_adv, feats_adv = net.gram_forward(adv_bx * 2 - 1)

        # backward
        optimizer.zero_grad()

        loss_reg = F.cross_entropy(logits_reg, by)
#         loss_adv = F.cross_entropy(logits_adv, by)
        
        loss_gram = 10**-3 * F.relu(10**4 - style_loss(feats_reg, feats_adv))
        loss = loss_reg + loss_gram
                                
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        
        # exponential moving average
        loss_avg = loss_avg * 0.8 + float(loss) * 0.2
        loss_gram_avg = loss_gram_avg * 0.8 + float(loss_gram) * 0.2
    
    state['train_loss'] = loss_avg
    state["gram_train_loss"] = loss_gram_avg
    
    print("Train Loss:", state["train_loss"])
    print("Train Gram: ", state["gram_train_loss"])

# test function
def test(detector = None):
    net.eval()
    loss_avg, loss_reg_avg, loss_adv_avg, loss_gram_avg, auroc_avg = 0.0, 0.0, 0.0, 0.0, 0.0
    loss, loss_reg, loss_adv, loss_gram = 0.0, 0.0, 0.0, 0.0
    correct = 0
    with torch.no_grad():
        for data, target in tqdm(test_loader):
            data, target = data.cuda(), target.cuda()
            
            adv_data = adversary(net, data, target)

            # forward
            output = net(data * 2 - 1)
#             adv_output = net(adv_data * 2 - 1)
            
#             loss_reg = F.cross_entropy(output, target)
#             loss_adv = F.cross_entropy(adv_output, target)
            
#             loss = loss_reg
            
            auroc = detector.compute_ood_deviations_batch(adv_data * 2 -1)
            auroc = auroc["AUROC"]
            
#             loss_gram = gram_margin_loss(feats_reg, feats_adv, margin=10e4)
            
            # accuracy
            pred = output.data.max(1)[1]
            correct += pred.eq(target.data).sum().item()

            # test loss average
#             loss_reg_avg += float(loss_reg)
#             loss_adv_avg += float(loss_adv)
#             loss_avg += float(loss)
#             loss_gram_avg += float(loss_gram)
            auroc_avg += auroc
            
            
    state['test_loss_reg'] = loss_reg_avg / len(test_loader)
    state['test_loss_adv'] = loss_adv_avg / len(test_loader)
    state['test_loss'] = loss_avg / len(test_loader)
    state['test_accuracy'] = correct / len(test_loader.dataset)
    state['gram_loss'] = loss_gram_avg / len(test_loader)
    state["gram_auroc"] = auroc_avg / len(test_loader)

In [ ]:
# Make save directory
if not os.path.exists(args.save):
    os.makedirs(args.save)
if not os.path.isdir(args.save):
    raise Exception('%s is not a dir' % args.save)

with open(os.path.join(args.save, args.dataset + '_' + args.model +
                                  '_baseline_training_results.csv'), 'w') as f:
    f.write('epoch,time(s),train_loss,test_loss,test_error(%),gram_auroc\n')

print('Beginning Training!\n')

# Main loop
for epoch in range(start_epoch, args.epochs):
    state['epoch'] = epoch

    begin_epoch = time.time()
    
    print("1. Training")
    train()
    print("2. Initializing Detector")
    
    if epoch % 5 == 0:
        net.eval()
        detector = Detector(net, data_train, data_test, args.test_bs, pbar=None)
        print("3. Testing")
        try:
            test(detector)
        except Exception as e:
            print("Failed test")
            print(e)

        # Save model
        torch.save(net.state_dict(),
                   os.path.join(args.save, args.dataset + '_' + args.model +
                                '_baseline_epoch_' + str(epoch) + '.pt'))
        # Let us not waste space and delete the previous model
        prev_path = os.path.join(args.save, args.dataset + '_' + args.model +
                                 '_baseline_epoch_' + str(epoch - 1) + '.pt')
        if os.path.exists(prev_path): os.remove(prev_path)

        # Show results

        with open(os.path.join(args.save, args.dataset + '_' + args.model +
                                          '_baseline_training_results.csv'), 'a') as f:
            f.write('%03d,%05d,%0.6f,%0.5f,%0.2f,%0.2f\n' % (
                (epoch + 1),
                time.time() - begin_epoch,
                state['train_loss'],
                state['test_loss'],
                100 - 100. * state['test_accuracy'],
                state["gram_auroc"],
            ))

        # # print state with rounded decimals
#         print({k: round(v, 4) if isinstance(v, float) else v for k, v in state.items()})

        print('Epoch {0:3d} | Time {1:5d} | Train Loss {2:.4f} | Test Loss {3:.3f} | Test Error {4:.2f} | Gram Auroc {5:.2f}'.format(
            (epoch + 1),
            int(time.time() - begin_epoch),
            state['train_loss'],
            state['test_loss'],
            100 - 100. * state['test_accuracy'],
            state["gram_auroc"])
        )


Beginning Training!

1. Training



Train Loss: 1.5973451894495407
Train Gram:  8.790877608043732e-19
2. Initializing Detector
3. Testing



Epoch   1 | Time   394 | Train Loss 1.5973 | Test Loss 0.000 | Test Error 65.03 | Gram Auroc 0.34
1. Training



Train Loss: 1.2572372374977823
Train Gram:  0.0
2. Initializing Detector
1. Training



Train Loss: 0.9250068469684873
Train Gram:  0.0
2. Initializing Detector
1. Training


...


Train Loss: 1.2138326168413032
Train Gram:  0.0
2. Initializing Detector
1. Training



Train Loss: 0.7696153412001018
Train Gram:  0.0
2. Initializing Detector
1. Training



Train Loss: 0.705399299077493
Train Gram:  0.0
2. Initializing Detector
1. Training



Train Loss: 0.590847807333346
Train Gram:  0.0
2. Initializing Detector
3. Testing



Epoch 181 | Time   372 | Train Loss 0.5908 | Test Loss 0.000 | Test Error 22.92 | Gram Auroc 0.81
1. Training
